In [19]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/08 01:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Set working directory

In [4]:
%cd ../data

/Users/lorenz/repos/data_engineering_zoomcamp/week_5_batch_processing/data


In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-16 21:13:50--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.84.132
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.84.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  35.6MB/s    in 21s     

2022-02-16 21:14:11 (34.4 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


Analyse column types:
Get first 1000 rows, read into pandas and see schema and head.

In [14]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > fhvhv_head.csv

In [6]:
import pandas as pd
df_pandas = pd.read_csv('fhvhv_head.csv')
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [8]:
df_pandas.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN


hvfhs_license_num: **String**  
dispatching_base_num: **String**  
pickup_datetime: **Datetime**  
dropoff_datetime: **Datetime**  
PULocationID: **Int**  
DOLocationID: **Int**  
SR_Flag: **Bool**  
  
According to the official data dictionary, SR_Flag is boolean:
https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_hvfhs.pdf

In [11]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.BooleanType(), True)
])

In [13]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('../data/fhvhv_tripdata_2021-01.csv')

In [14]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,BooleanType,true)))

In [15]:
df = df.repartition(24)

In [ ]:
df.write.parquet('../data/fhvhv/2021/01/')

In [17]:
df = spark.read.parquet('../data/fhvhv/2021/01/')

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: boolean (nullable = true)



```
SELECT pickup_datetime, 
       dropoff_datetime, 
       PULocationID, 
       DOLocationID 
FROM df 
WHERE hvfhs_license_num =  HV0003
```

The spark code below is equivalent to the sql statement above.

In [25]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 02:31:31|2021-01-01 02:39:30|         168|          41|
|2021-01-03 14:15:23|2021-01-03 14:21:05|         145|         226|
|2021-01-04 07:15:12|2021-01-04 07:28:18|          49|          25|
|2021-01-02 17:29:50|2021-01-02 17:37:02|         215|         130|
|2021-01-03 12:18:07|2021-01-03 12:25:59|         226|         145|
|2021-01-01 10:45:44|2021-01-01 10:53:22|          14|         228|
|2021-01-03 12:43:02|2021-01-03 12:49:29|         244|         243|
|2021-01-03 01:53:03|2021-01-03 02:00:44|          60|         182|
|2021-01-01 07:46:59|2021-01-01 07:52:04|         213|         213|
|2021-01-02 11:45:37|2021-01-02 12:10:50|         244|          82|
|2021-01-04 10:36:30|2021-01-04 10:55:28|          42|         230|
|2021-01-01 02:44:53|2021-01-01 02:58:22|       

You could insert user defined functions (udf) into spark:
1. create python function (which can be tested)
2. make function an udf in spark

In [22]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [23]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

Use udf in spark query:  
_See also column conversion to date ``F.to_date``_

In [24]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|         256|          61|
|  s/b3d| 2021-01-01|  2021-01-01|         168|          41|
|  e/acc| 2021-01-03|  2021-01-03|         145|         226|
|  e/a39| 2021-01-04|  2021-01-04|          49|          25|
|  e/9ce| 2021-01-01|  2021-01-01|          26|          26|
|  e/9ce| 2021-01-02|  2021-01-03|         138|         265|
|  e/b3b| 2021-01-02|  2021-01-02|         215|         130|
|  e/b3b| 2021-01-03|  2021-01-03|         226|         145|
|  a/b37| 2021-01-01|  2021-01-01|          14|         228|
|  e/9ce| 2021-01-02|  2021-01-02|         129|          83|
|  e/b3b| 2021-01-03|  2021-01-03|         244|         243|
|  e/9ce| 2021-01-02|  2021-01-02|          47|         159|
|  e/b3f| 2021-01-03|  2021-01-03|          60|         182|
|  s/acd| 2021-01-01|  2

In [26]:
spark.stop()